<a href="https://colab.research.google.com/github/PacktPublishing/Machine-Learning-for-Time-Series-with-Python/blob/master/chapter7/Silverkite.ipynb" target="_parent\"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install greykite

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
  Using cached holidays-0.13-py3-none-any.whl.metadata (11 kB)
Using cached holidays-0.13-py3-none-any.whl (172 kB)
  Attempting uninstall: holidays
    Found existing installation: holidays 0.73
    Uninstalling holidays-0.73:
      Successfully uninstalled holidays-0.73
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
featuretools 1.31.0 requires holidays>=0.17, but you have holidays 0.13 which is incompatible.
featuretools 1.31.0 requires pandas>=2.0.0, but you have pandas 1.5.3 which is incompatible.
prophet 1.1.6 requires holidays<1,>=0.25, but you have holidays 0.13 which is incompatible.


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.style.use('seaborn-v0_8-whitegrid')

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = "17"

In [3]:
import pandas as pd

owid_covid = pd.read_csv("https://covid.ourworldindata.org/data/owid-covid-data.csv")
owid_covid["date"] = pd.to_datetime(owid_covid["date"])

In [4]:
owid_covid.location.unique()

array(['Afghanistan', 'Africa', 'Albania', 'Algeria', 'American Samoa',
       'Andorra', 'Angola', 'Anguilla', 'Antigua and Barbuda',
       'Argentina', 'Armenia', 'Aruba', 'Asia', 'Australia', 'Austria',
       'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados',
       'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan',
       'Bolivia', 'Bonaire Sint Eustatius and Saba',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Cayman Islands', 'Central African Republic', 'Chad', 'Chile',
       'China', 'Colombia', 'Comoros', 'Congo', 'Cook Islands',
       'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Curacao',
       'Cyprus', 'Czechia', 'Democratic Republic of Congo', 'Denmark',
       'Djibouti', 'Dominica', 'Dominican Republic', 'East Timor',
       'Ecuador', 'Egypt', 'El Salvador', 'England', 'Equatorial Guine

In [5]:
owid_covid.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-05,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-06,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-07,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-08,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-09,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN


In [6]:
df = owid_covid[owid_covid.location == "France"].set_index("date", drop=True).resample('D').interpolate(method='linear')

/tmp/ipykernel_104968/2325790796.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = owid_covid[owid_covid.location == "France"].set_index("date", drop=True).resample('D').interpolate(method='linear')


In [7]:
from greykite.framework.templates.autogen.forecast_config import (
    ForecastConfig, MetadataParam
)

metadata = MetadataParam(
    time_col="date",
    value_col="new_cases",
    freq="D"
)

In [8]:
import warnings
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum

forecaster = Forecaster()

warnings.filterwarnings("ignore", category=UserWarning)
result = forecaster.run_forecast_config(
    df=df.reset_index(),
    config=ForecastConfig(
        model_template=ModelTemplateEnum.SILVERKITE_DAILY_90.name,
        forecast_horizon=90,
        coverage=0.95,
        metadata_param=metadata,
    )
)

/root/miniconda3/envs/TimeSeries/lib/python3.12/site-packages/greykite/common/time_properties.py:382: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[value_col].replace([np.inf, -np.inf], np.nan, inplace=True)
/root/miniconda3/envs/TimeSeries/lib/python3.12/site-packages/greykite/common/time_properties.py:382: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the i

Fitting 3 folds for each of 4 candidates, totalling 12 fits


/root/miniconda3/envs/TimeSeries/lib/python3.12/site-packages/greykite/algo/uncertainty/conditional/estimate_distribution.py:96: FutureWarning: A grouping was used that is not in the columns of the DataFrame and so was excluded from the result. This grouping will be included in a future version of pandas. Add the grouping as a column of the DataFrame to silence this warning.
  ecdf_df_overall = df.groupby([True]*len(df), as_index=False).agg(agg_dict)
/root/miniconda3/envs/TimeSeries/lib/python3.12/site-packages/greykite/algo/changepoint/adalasso/changepoint_detector.py:332: FutureWarning: Day.delta is deprecated and will be removed in a future version. Use pd.Timedelta(obj) instead
  no_changepoint_proportion_from_end = to_offset(no_changepoint_distance_from_end).delta / data_length
/root/miniconda3/envs/TimeSeries/lib/python3.12/site-packages/greykite/algo/changepoint/adalasso/changepoint_detector.py:337: FutureWarning: Day.delta is deprecated and will be removed in a future version. 

In [9]:
forecast = result.forecast
forecast.plot().show(renderer="colab")  # leave out the renderer argument if you are not on google colab!

In [10]:
forecast.df.head().round(2)

,ts,actual,forecast,forecast_lower,forecast_upper
0,2020-01-05,0.0,8079.99,-351740.55,367900.53
1,2020-01-06,0.0,3494.65,-30431.13,37420.43
2,2020-01-07,0.0,5046.40,-28049.28,38142.09
3,2020-01-08,0.0,10673.75,-24322.62,45670.13
4,2020-01-09,0.0,3766.36,-36329.89,43862.61


In [11]:
from collections import defaultdict

backtest = result.backtest
backtest_eval = defaultdict(list)
for metric, value in backtest.train_evaluation.items():
    backtest_eval[metric].append(value)
    backtest_eval[metric].append(backtest.test_evaluation[metric])
metrics = pd.DataFrame(backtest_eval, index=["train", "test"]).T
metrics.head()

,train,test
CORR,0.868565,NaN
R2,0.754404,0.0
MSE,5115672218.46764,844099667.08332
RMSE,71523.927594,29053.393383
MAE,24074.286882,18897.83913


In [12]:
model = result.model
future_df = result.timeseries.make_future_dataframe(
    periods=4,
    include_history=False
)
model.predict(future_df)

/root/miniconda3/envs/TimeSeries/lib/python3.12/site-packages/greykite/sklearn/transform/null_transformer.py:177: RuntimeWarning:

Input data has many null values. Missing 100.00% of one input.



,ts,forecast,quantile_summary,err_std,forecast_lower,forecast_upper
0,2024-08-05,-5333.188651,"(-39345.69151449316, 28679.314212117784)",17353.636665,-39345.691514,28679.314212
1,2024-08-06,3158.213884,"(-30049.726712594907, 36366.154481021156)",16943.138169,-30049.726713,36366.154481
2,2024-08-07,-11923.660962,"(-47031.3577874793, 23184.0358634055)",17912.419362,-47031.357787,23184.035863
3,2024-08-08,-10528.743870,"(-50701.00887124002, 29643.521130552243)",20496.430199,-50701.008871,29643.521131


In [13]:
model.predict(future_df)

/root/miniconda3/envs/TimeSeries/lib/python3.12/site-packages/greykite/sklearn/transform/null_transformer.py:177: RuntimeWarning:

Input data has many null values. Missing 100.00% of one input.



,ts,forecast,quantile_summary,err_std,forecast_lower,forecast_upper
0,2024-08-05,-5333.188651,"(-39345.69151449316, 28679.314212117784)",17353.636665,-39345.691514,28679.314212
1,2024-08-06,3158.213884,"(-30049.726712594907, 36366.154481021156)",16943.138169,-30049.726713,36366.154481
2,2024-08-07,-11923.660962,"(-47031.3577874793, 23184.0358634055)",17912.419362,-47031.357787,23184.035863
3,2024-08-08,-10528.743870,"(-50701.00887124002, 29643.521130552243)",20496.430199,-50701.008871,29643.521131
